In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt 
import numpy as np

#helper function to get responses of the api and store each layer in a dictionary
def get_responses(*args):
    list_of_responses={}
    for parm in args:
        url = f"https://emotional.byteroad.net/collections/{parm}/items?f=json&lang=en-US&limit=10000&skipGeometry=false&offset=0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data_dict = json.loads(response.content)
            list_of_responses[parm]=data_dict
        else:
            print("Error: API request unsuccessful.")
    return list_of_responses

In [2]:
from branca.element import MacroElement
from jinja2 import Template

class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('layeradd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('layerremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)    

In [3]:
import folium
from folium import plugins
from folium import Map, FeatureGroup, Marker, LayerControl
import branca.colormap as cm
import branca

#dynamic legend values
column_names = ['av_age_bui', 'bui_repair', 'walkabil', 'altimetry', 'beds_tour', 'ff_dens', 'ndvi', 'noise', 'pm2_5','temp', 'vul_heat','vul_flood', 'no2', 'g_spa_dist']

data = {column_name: [] for column_name in column_names}

responses = [
    get_responses('lisbon_average_age_buildings')['lisbon_average_age_buildings']['features'],
    get_responses('lisbon_building_repair_needs_ratio')['lisbon_building_repair_needs_ratio']['features'],
    get_responses('lisbon_walkability')['lisbon_walkability']['features'],
    get_responses('lisbon_altimetry')['lisbon_altimetry']['features'],
    get_responses('lisbon_beds_tourist_accomodations')['lisbon_beds_tourist_accomodations']['features'],
    get_responses('lisbon_fast_food_outlets_density')['lisbon_fast_food_outlets_density']['features'],
    get_responses('lisbon_ndvi')['lisbon_ndvi']['features'],
    get_responses('lisbon_noise')['lisbon_noise']['features'],
    get_responses('lisbon_pm2_5')['lisbon_pm2_5']['features'],
    get_responses('lisbon_temperature')['lisbon_temperature']['features'],
    get_responses('lisbon_vulnerability_excessive_heat')['lisbon_vulnerability_excessive_heat']['features'],
    get_responses('lisbon_vulnerability_flash_floods')['lisbon_vulnerability_flash_floods']['features'],
    get_responses('lisbon_no2')['lisbon_no2']['features'],
    get_responses('lisbon_distance_green_spaces')['lisbon_distance_green_spaces']['features']
]
    
for response_data in responses:
    for feature in response_data:
        properties = feature['properties']
        for column_name in column_names:
            value = properties.get(column_name, None)
            data[column_name].append(value)
        
df = pd.DataFrame(data)

#print(df)

min_values = df.min(skipna=True)
max_values = df.max(skipna=True)

num_groups = 4
group_boundaries = {}
for column in df.columns:
    min_value = min_values[column]
    max_value = max_values[column]
    
    if min_value is None or max_value is None:
        continue  # Skip columns with missing or null values
        
    group_size = (max_value - min_value) / num_groups
    boundaries = np.arange(min_value, max_value + group_size, group_size)
    group_boundaries[column] = boundaries

# Print the group boundaries for each column
for column, boundaries in group_boundaries.items():
    print(f"Group boundaries for column '{column}':")
    print(boundaries)
    print()

Group boundaries for column 'av_age_bui':
[   0.          504.03571428 1008.07142857 1512.10714285 2016.14285714]

Group boundaries for column 'bui_repair':
[  0.  25.  50.  75. 100.]

Group boundaries for column 'walkabil':
[-1.66275125 -0.57069835  0.52135455  1.61340745  2.70546036]

Group boundaries for column 'altimetry':
[  0.          53.51501464 107.03002928 160.54504392 214.06005856]

Group boundaries for column 'beds_tour':
[   91.    7774.75 15458.5  23142.25 30826.  ]

Group boundaries for column 'ff_dens':
[0.         0.01424723 0.02849446 0.04274169 0.05698892]

Group boundaries for column 'ndvi':
[-0.55552034 -0.20500311  0.14551412  0.49603136  0.84654859]

Group boundaries for column 'noise':
[43.56842365 55.52745388 67.48648412 79.44551435 91.40454459]

Group boundaries for column 'pm2_5':
[ 6.07382588  7.30191495  8.53000401  9.75809307 10.98618214]

Group boundaries for column 'temp':
[21.09115951 21.67910915 22.26705879 22.85500842 23.44295806 24.0309077 ]

Group b

In [4]:
def hexmap(responses, column_name, color, feature_group):
    for response in responses:
        for feature in response:
            properties = feature['properties']
            if column_name in properties:
                value = properties[column_name]
                geoj = folium.GeoJson(feature, style_function=color)
                folium.features.GeoJsonPopup(fields=[column_name], aliases=['Mean Value'], labels=True, localize=True).add_to(geoj)
                feature_group.add_child(geoj)


#av_age_bui

color_av_age_bui = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['av_age_bui'] < 0 else 
    '#FFFFCC' if 0 <= x['properties']['av_age_bui'] < 504.04 else 
    '#FFCC99' if 504.04 <= x['properties']['av_age_bui'] < 1008.07 else 
    '#FF9933' if 1008.07 <= x['properties']['av_age_bui'] < 1512.11 else
    '#FF6633', 'color': 'white', 'weight': 1, 'fillOpacity': 0.7
}

fg1 = folium.FeatureGroup(name='Average age of buildings', overlay=True, control=False)
hexmap(responses, 'av_age_bui', color_av_age_bui, fg1)
scale_fg1 = branca.colormap.StepColormap(['#FFFFCC','#FFCC99','#FF9933','#FF6633'],index=[0,504.04,1008.07,1512.11,2016.14], vmin=0, vmax=2016.14, caption='Average age of buildings')       

#buil_repai 

color_repai = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['bui_repair'] < 0 else 
    '#FFFFCC' if 0 <= x['properties']['bui_repair'] < 25 else 
    '#FFCC99' if 25 <= x['properties']['bui_repair'] < 50 else 
    '#FF9933' if 50 <= x['properties']['bui_repair'] < 75 else
    '#FF6633' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg2 = folium.FeatureGroup(name = 'Buildings with repair needs ratio', overlay=True, control=False)
hexmap(responses, 'bui_repair', color_repai, fg2)
scale_fg2 = branca.colormap.StepColormap(['#FFFFCC','#FFCC99','#FF9933','#FF6633'],index=[0,25,50,75,100], vmin=0, vmax=100, caption='Buildings with repair needs ratio')       


#walkabilit

color_walk = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['walkabil'] < -1.66 else 
    '#FFA500' if -1.66 <= x['properties']['walkabil'] < -0.57 else 
    '#FFFF00' if -0.57 <= x['properties']['walkabil'] < 0.52 else 
    '#7FFF00' if 0.52 <= x['properties']['walkabil'] < 1.61 else
    '#228B22' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg3 = folium.FeatureGroup(name='Walkability index', overlay=True, control=False)
hexmap(responses, 'walkabil', color_walk, fg3)
scale_fg3 = branca.colormap.StepColormap(['#FFA500','#FFFF00','#7FFF00','#228B22'],index=[-1.66,-0.57,0.52,1.61,2.71], vmin=-1.66, vmax=2.71, caption='Walkability index')

#altimetry

color_altitude = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['altimetry'] < 0 else 
    '#000000' if 0 <= x['properties']['altimetry'] < 53.52 else 
    '#3366CC' if 53.52 <= x['properties']['altimetry'] < 107.03 else 
    '#66CCFF' if 107.03 <= x['properties']['altimetry'] < 160.55 else
    '#99FFFF' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg4 = folium.FeatureGroup(name = 'Altimery', overlay=True, control=False)
hexmap(responses, 'altimetry', color_altitude, fg4)
scale_fg4 = branca.colormap.StepColormap(['#000000','#3366CC','#66CCFF','#99FFFF'],index=[0,53.52,107.03,160.55,214.06], vmin=0, vmax=214.06, caption='Altimetry')

#beds_tour_

color_beds = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['beds_tour'] < 91 else 
    '#FFCC99' if 91 <= x['properties']['beds_tour'] < 7774.75 else 
    '#FF9933' if 7774.75 <= x['properties']['beds_tour'] < 15458.5 else 
    '#CC6600' if 15458.5 <= x['properties']['beds_tour'] < 23142.25 else
    '#663300' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg5 = folium.FeatureGroup(name = 'Beds/customers in tourist accomodations', overlay=True, control=False)
hexmap(responses,'beds_tour',color_beds,fg5)
scale_fg5 = branca.colormap.StepColormap(['#FFCC99', '#FF9933', '#CC6600', '#663300'],index=[91,7774.75,15458.5,23142.25,30826], vmin=91, vmax=30826, caption='Beds/customers in tourist accomodations')


#ff_out_den

color_ff = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['ff_dens'] < 0 else 
    '#FFCC99' if 0 <= x['properties']['ff_dens'] < 0.01 else 
    '#FF9933' if 0.01 <= x['properties']['ff_dens'] < 0.03 else 
    '#CC6600' if 0.02 <= x['properties']['ff_dens'] < 0.04 else
    '#663300' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg6 = folium.FeatureGroup(name = 'Density of fast food outlets', overlay=True, control=False)
hexmap(responses, 'ff_dens', color_ff, fg6)
scale_fg6 = branca.colormap.StepColormap(['#FFCC99', '#FF9933', '#CC6600', '#663300'],index=[0,0.01,0.03,0.04,0.06], vmin=0, vmax=0.06, caption='Density of fast food outlets')


#ndvi

color_ndvi = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['ndvi'] < -0.56 else 
    '#A2FF9E' if -0.56 <= x['properties']['ndvi'] < -0.21 else 
    '#60C957' if -0.21 <= x['properties']['ndvi'] < 0.15 else 
    '#008000'if 0.15 <= x['properties']['ndvi'] < 0.50 else
    '#004D00','color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg7 = folium.FeatureGroup(name = 'Normalized Difference Vegetation Index (NDVI)', overlay=True, control=False)
hexmap(responses, 'ndvi', color_ndvi, fg7)
scale_fg7 = branca.colormap.StepColormap(['#A2FF9E','#60C957','#008000','#004D00'],index=[-0.56, -0.21, 0.15, 0.50, 0.85], vmin=-0.56, vmax=0.85, caption='Normalized Difference Vegetation Index (NDVI)')


#noise

color_noise = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['noise'] is not None and x['properties']['noise'] < 43.57 else
    '#FFFFCC' if x['properties']['noise'] is not None and 43.57 <= x['properties']['noise'] < 55.53 else             
    '#FFCC99' if x['properties']['noise'] is not None and 55.53 <= x['properties']['noise'] < 67.49 else
    '#FF9933' if x['properties']['noise'] is not None and 67.49 <= x['properties']['noise'] < 79.45 else
    '#FF6633' if x['properties']['noise'] is not None else None,
    'color': 'white',
    'weight': 1,
    'fillOpacity': '0.7'}

fg8 = folium.FeatureGroup(name = 'Noise level', overlay=True, control=False)
hexmap(responses, 'noise', color_noise, fg8)
scale_fg8 = branca.colormap.StepColormap(['#FFFFCC','#FFCC99','#FF9933','#FF6633'],index=[43.57,55.53,67.49,79.45,91.40], vmin=43.57, vmax=91.40, caption='Noise level')


#pm25

color_pm25 = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['pm2_5'] < 6.07 else 
    '#E6F5FF' if 6.07 <= x['properties']['pm2_5'] < 7.30 else 
    '#B2D4FF' if 7.30 <= x['properties']['pm2_5'] < 8.53 else 
    '#8A7FFF' if 8.53 <= x['properties']['pm2_5'] < 9.76 else
    '#9370DB' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg9 = folium.FeatureGroup(name = 'Particulate Matter (PM2.5)', overlay=True, control=False)
hexmap(responses, 'pm2_5', color_pm25, fg9)
scale_fg9 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB'],index=[6.07,7.30,8.53,9.76,10.99], vmin=6.07, vmax=10.99, caption='Particulate Matter (PM25)')


#mean_temp

color_temp = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['temp'] < 21.09 else 
    '#0066CC' if 21.09 <= x['properties']['temp'] < 21.68 else 
    '#66CCFF' if 21.68 <= x['properties']['temp'] < 22.27 else 
    '#FFCC00' if 22.27 <= x['properties']['temp'] < 22.86 else
    '#FF6600' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg10 = folium.FeatureGroup(name = 'Mean temperature', overlay=True, control=False)
hexmap(responses, 'temp', color_temp, fg10)
scale_fg10 = branca.colormap.StepColormap(['#0066CC','#66CCFF','#FFCC00','#FF6600'],index=[21.09,21.68,22.27,22.86,24.03], vmin=21.09, vmax=24.03, caption='Mean temperature')


#ex_heat_vu

color_exheat = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['vul_heat'] < 0.34 else 
    '#FFA500' if 0.34 <= x['properties']['vul_heat'] < 0.44 else 
    '#FF8C00' if 0.44 <= x['properties']['vul_heat'] < 0.54 else 
    '#FF4500' if 0.54 <= x['properties']['vul_heat'] < 0.63 else
    '#FF0000' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg11 = folium.FeatureGroup(name = 'Extreme heat vulnerability', overlay=True, control=False)
hexmap(responses, 'vul_heat', color_exheat, fg11)
scale_fg11 = branca.colormap.StepColormap(['#FFA500','#FF8C00','#FF4500','#FF0000'],index=[0.34,0.44,0.54,0.63,0.72], vmin=0.35, vmax=0.72, caption='Extreme heat vulnerability')

#ffloods_vu

color_floods = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['vul_flood'] < 0.19 else 
    '#66FFFF' if 0.19 <= x['properties']['vul_flood'] < 0.27 else 
    '#3366FF' if 0.27 <= x['properties']['vul_flood'] < 0.34 else 
    '#0033CC' if 0.34 <= x['properties']['vul_flood'] < 0.41 else
    '#000066' , 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg12 = folium.FeatureGroup(name = 'Vulnerability to flash floods index', overlay=True, control=False)
hexmap(responses, 'vul_flood', color_floods, fg12)
scale_fg12 = branca.colormap.StepColormap(['#66FFFF','#3366FF','#0033CC','#000066'],index=[0.19,0.27,0.34,0.41,0.49], vmin=0.19, vmax=0.49, caption='Vulnerability to flash floods index')

#no2

color_no2 = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['no2'] < 7.90 else 
    '#E6F5FF' if 7.90 <= x['properties']['no2'] < 9.97 else 
    '#B2D4FF' if 9.97 <= x['properties']['no2'] < 12.03 else 
    '#8A7FFF' if 12.03 <= x['properties']['no2'] < 14.09 else
    '#9370DB', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg13 = folium.FeatureGroup(name='Nitrogen Dioxide (NO2)', overlay=True, control=False)
hexmap(responses, 'no2', color_no2, fg13)
scale_fg13 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB'], index=[7.90,9.97,12.03,14.09,16.15], vmin=7.90, vmax=16.15, caption='NO2 (μg/m³)')

#g_spa_dist

color_g_spa_dist = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['g_spa_dist'] < 0 else 
    '#FF0000' if 0 <= x['properties']['g_spa_dist'] < 0.25 else 
    '#FFA500' if 0.25 <= x['properties']['g_spa_dist'] < 0.50 else 
    '#FFFF00' if 0.50 <= x['properties']['g_spa_dist'] < 0.75 else
    '#00FF00', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg14 = folium.FeatureGroup(name='Distance to green spaces', overlay=True, control=False)
hexmap(responses, 'g_spa_dist', color_g_spa_dist, fg14)
scale_fg14 = branca.colormap.StepColormap(['#FF0000','#FFA500','#FFFF00','#00FF00'], index=[0,0.25,0.50,0.75,1], vmin=0, vmax=1, caption='Distance to green spaces')


In [5]:
# Add each layer to the map and save it as a separate HTML file

for i in range(1, 15):
    # Create a new map object for each iteration
    m = folium.Map(location=[38.736946, -9.142685], control_scale=True, zoom_start=12, tiles=None)
    
    base_map1 = folium.FeatureGroup(name='OpenStreetMap', overlay=False)
    folium.TileLayer(tiles='OpenStreetMap').add_to(base_map1)
    # Add satellite imagery basemap
    base_map2 = folium.FeatureGroup(name="Esri World Imagery", overlay=False)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite'
    ).add_to(base_map2)

    base_map1.add_to(m)
    base_map2.add_to(m)
    
    fg_name = "fg" + str(i)
    fg = globals()[fg_name]
    m.add_child(fg)
    
    scale_name = "scale_fg" + str(i)
    scale = globals()[scale_name]
    m.add_child(scale)
    m.add_child(BindColormap(fg, scale))

    # Add LayerControl to toggle between basemaps
    folium.LayerControl(position='bottomleft').add_to(m)
    
    map_fname = f'Output_Lisbon_Physical{i}.html'
    m.save(map_fname)